In [2]:
import glob
import os
import pandas as pd
import numpy as np

In [3]:
files = glob.glob('training_data\\20160617\\*')
sfiles = sorted(files, key=os.path.getctime)

In [4]:
trains = pd.DataFrame()
for sfile in sfiles:
    train = pd.read_csv(sfile)
    trains = pd.concat([trains, train])

In [5]:
trains = trains.reset_index().drop('index',axis=1)

In [6]:
rows = trains.shape[0]
prev_score = 0
prev_row = 0
kills = []
for row in range(rows):
    if trains.ix[row, 'p1_score'] != prev_score:
        kills.append([prev_row, row])
        prev_score = trains.ix[row, 'p1_score']
        prev_row = row + 1


c:\program files\anaconda3\envs\sb\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [7]:
trains['reward_adjusted'] = 0

In [8]:
td_lookback = 25
decay = 0.9
reward = 1

for kill in kills:
    killpoint = kill[1]
    exponent = td_lookback - 1
    for row in range(killpoint - td_lookback, killpoint):
        trains.ix[row, 'reward_adjusted'] = reward * decay ** exponent
        exponent -= 1

In [9]:
trains_backup = trains.copy()

In [10]:
drop_cols = ['episode', 'reward', 'p1_score', 'p2_score', 'reward_adjusted',
             'p1_x_delta', 'p2_x_delta', 'p1_y_delta', 'p2_y_delta',
            'p1h', 'p2h']
y = trains_backup['reward_adjusted']
for item in drop_cols:
    trains_backup.drop(item,inplace=True,axis=1)

In [11]:
trains_backup.head(2)

,p1_x,p1_y,p2_x,p2_y,right_of,left_of,above,below,x_clear,y_clear,cd1,cd2,aim_down,aim_left,aim_up,aim_right,action
0,2.0,-3.7,-8.0,5.3,0,1,0,1,0,0,0,0,1,0,0,0,1
1,2.0,-3.7,-8.0,5.3,0,1,0,1,0,0,0,0,1,0,0,0,5


In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.layers import Dropout


def keras_model(inputs, dropouts=0.2, opti=1e-6, clipping=0.1, clipval=10):
    keras_model = Sequential()
    keras_model.add(Dense(128, input_dim=inputs, activation='relu'))
    keras_model.add(Dropout(dropouts))
    keras_model.add(Dense(128, activation='relu'))
    keras_model.add(Dropout(dropouts))
    keras_model.add(Dense(1))
    adam = Adam(lr=opti, clipnorm=clipping, clipvalue=clipval)
    keras_model.compile(loss='mse', optimizer='adam')
    return keras_model


Using Theano backend.


In [4]:
input_shape = trains_backup.shape[1]


Qas = keras_model(inputs=input_shape, dropouts=0.2, opti=1e-6, clipping=0.05, clipval=10)

In [13]:
training_set = np.array(trains_backup)
Y = np.array(y)

In [14]:
Qas.fit(training_set, Y, epochs=50, verbose=1)

c:\program files\anaconda3\envs\sb\lib\site-packages\keras\models.py:851: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/50
170000/170000 [==============================] - 9s - loss: 0.0673     
Epoch 2/50
170000/170000 [==============================] - 17s - loss: 0.0108    
Epoch 3/50
170000/170000 [==============================] - 26s - loss: 0.0107    
Epoch 4/50
170000/170000 [==============================] - 32s - loss: 0.0107    
Epoch 5/50
170000/170000 [==============================] - 32s - loss: 0.0106    
Epoch 6/50
170000/170000 [==============================] - 32s - loss: 0.0106    
Epoch 7/50
170000/170000 [==============================] - 34s - loss: 0.0106    
Epoch 8/50
170000/170000 [==============================] - 32s - loss: 0.0106    
Epoch 9/50
170000/170000 [==============================] - 32s - loss: 0.0106    
Epoch 10/50
170000/170000 [==============================] - 37s - loss: 0.0106    
Epoch 11/50
170000/170000 [==============================] - 36s - loss: 0.0106    
Epoch 12/50
170000/170000 [==============================] - 32s - loss: 0.0106    
E

In [5]:
import h5py
Qas.save_weights("SB-Qas-v0.keras")

In [32]:
datapoint = env.create_predictions(datapoint)
action_rewards = Qas.predict(predictions

SyntaxError: unexpected EOF while parsing (<ipython-input-32-6fec285e563b>, line 2)

In [ ]:
#action = np.where(advantage == np.amax(advantage))[0][0]
        #action = np.where(action_rewards == np.amax(action_rewards))[0][0] # greedy action